In [1]:
import pandas as pd
import numpy as np
import cv2
import glob

In [86]:
###  Checking if there is any other file except jpg in each folder

face_expressions_folders_path = glob.glob("Dataset/*")
for path in face_expressions_folders_path:
    images_path = glob.glob(path+"/*")
    for image in images_path:
        if(image.endswith('jpg')):
            pass
        else:
            print(image)


In [101]:
###  Displaying a random image

face_expression_images = glob.glob(face_expressions_folders_path[0]+"/*")
img = cv2.imread(face_expression_images[0])
cv2.imshow("img",img)
cv2.waitKey()
cv2.destroyAllWindows()

In [93]:
X = []              ##  Initialising an empty list to collect array of images
y = []              ##  Initialising an empty list to collect target values of each image

for path in face_expressions_folders_path:                            # Reading each folder's path
    face_expression_images = glob.glob(path+"/*")                     # For each folder path, reading the path of all the images in that folder
    for image in face_expression_images:                              # Looping through each image in every folder
        img = cv2.imread(image)                                            
        img = cv2.resize(img,(48,48))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = gray.flatten()                                         # Flattening the image and converting it to 2D array
        gray = gray/255
        X.append(gray)
        y.append(path.split('\\')[-1])                                # Appending the name of facial expression against each image by trimming it from folder path

        

In [102]:
from collections import Counter
image_count = Counter(y)           # Getting image count 
image_count

Counter({'angry': 500,
         'disgust': 500,
         'fear': 500,
         'happy': 500,
         'sad': 500,
         'surprise': 500})

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score

In [152]:
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=10,train_size=0.75,test_size=0.25)

In [159]:
from sklearn.ensemble import RandomForestClassifier
#pca = PCA(0.9)
#X_train = pca.fit_transform(X_train)

#train_score = []
#test_score = []
#for i in range(25,100):
    #model = RandomForestClassifier(n_estimators=i)
    #model.fit(X_train, y_train)

    #X_test = pca.transform(X_test)
    #pred_test = model.predict(X_test)
    #train_score.append(model.score(X_train,y_train))
    #test_score.append(model.score(X_test,y_test))

In [178]:
model = RandomForestClassifier(n_estimators=79)
model.fit(X_train, y_train)
pred_test = model.predict(X_test)
print('Training score : ',(model.score(X_train,y_train)))
print('Testing score : ',(model.score(X_test,y_test)))

Training score :  1.0
Testing score :  0.37466666666666665


## Testing with webcam

In [209]:
vdo = cv2.VideoCapture(0)                   # Enabling webcam

faceclf = cv2.CascadeClassifier("D:/Trainings/python_with_ML_Ducat_INdustrial_training/resource/jupyter/DATASETS/haar/haarcascade_frontalface_default.xml")

while(True):
    flag, img = vdo.read()                                        ## Reading the image from webcam 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = cv2.flip(gray,1)
    img = cv2.flip(img,1)                                         ## Flipping the image read so that it can be displayed without mirror reflection
    
    if(flag==False):
        break
    
    faces = faceclf.detectMultiScale(gray,minNeighbors = 10, scaleFactor = 1.4)              ## Detecting the face from webcam stream
    
    for x,y,w,h in faces:                           ## Cropping out the face from the image
        face = gray[y:y+h,x:x+w]
        face = cv2.resize(face,(48,48))
        face = face.flatten()
        face = face/255
        pred = model.predict([face])                ## Predicting the image
        
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),2)                                     ## Marking face and writing the predicted expression on image
        cv2.putText(img,pred[0],(x,y+10),cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),2)
        
    cv2.imshow("image",img)
    k = cv2.waitKey(1)
    if (k == ord('a')):
        break
        
cv2.destroyAllWindows()
vdo.release()
    
    
    
    
    
    